#### Dependencies

In [519]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import time
import multiprocessing
from multiprocessing.shared_memory import SharedMemory
from dotenv import load_dotenv
import tqdm

from essentia.standard import (
    MonoLoader,
    Danceability,
    Spectrum,
    FrameCutter,
    Loudness,
    RhythmExtractor2013,
    KeyExtractor,
    Energy,
    TonalExtractor,
    Inharmonicity,
    MFCC,
    OnsetRate,
    SpectralCentroidTime,
    DynamicComplexity,
    SpectralPeaks,
    NoveltyCurve,
    Spectrum,
    FrameGenerator,
    Windowing,
    MelBands,
    BeatsLoudness,
    Beatogram,
    Meter,
    HumDetector,
)

#### Global Constants

In [520]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
CPU_THREADS = multiprocessing.cpu_count()

#### Data

In [521]:
songs_data = pd.read_csv('data/songs_final.csv')

#### Feature Extraction Functions

In [522]:
def create_spectrogram_image(spectrogram_db, sample_rate):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', fmax=11025)
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Mel-Spectrogram")
    plt.tight_layout()
    plt.show()
    plt.close()

In [523]:
def mp3_to_spectrogram(audio_path, sample_rate, create_image=False):
    mp3, _ = librosa.load(audio_path, sr=sample_rate)
    spectrogram = librosa.feature.melspectrogram(y=mp3, sr=sample_rate, n_mels=128, fmax=11025)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    if create_image:
        create_spectrogram_image(spectrogram_db, sample_rate)

    return spectrogram_db

In [524]:
def get_mel_bands(audio):
    spectrum = Spectrum()
    frame_generator = FrameGenerator(audio, frameSize=2048, hopSize=1024)
    window = Windowing(type='hann')

    mel_bands = MelBands(numberBands=40)
    mel_band_energies = []

    for frame in frame_generator:
        spec = spectrum(window(frame))
        mel_band_energies.append(mel_bands(spec))

    mel_band_energies = np.array(mel_band_energies)
    return mel_band_energies

In [525]:
def run_essentia_algorithms(audio44k, audio16k):
    _, mfcc_coeffs = MFCC(inputSize=len(audio16k))(audio16k)
    danceability_score = Danceability()(audio44k)
    loudness_score = Loudness()(audio16k)
    bpm, beat_positions, _, _, _ = RhythmExtractor2013(method="multifeature")(audio44k)
    key, scale, _ = KeyExtractor()(audio44k)
    energy_score = Energy()(audio16k)

    ### Chord Significances
    _, _, _, _, chords, _, _, _, _, _, _, _ = TonalExtractor()(audio44k)
    unique_chords, counts = np.unique(chords, return_counts=True)
    chords_significance = {chord: significance for (chord, significance) in zip(unique_chords, counts)}

    ### Inharmonicity
    frames = []
    frameCutter = FrameCutter()
    while True:
        frame = frameCutter(audio44k)
        if not len(frame):
            break
        frames.append(frame)
        
    spectrum_magnitudes = []
    for frame in frames:
        spectrum_magnitudes_frame = Spectrum()(frame)
        spectrum_magnitudes.append(spectrum_magnitudes_frame)
    spectrum_magnitudes = np.array(spectrum_magnitudes).flatten()
    
    frequencies, magnitudes = SpectralPeaks()(audio44k)
    hnr_score = None
    if frequencies[0]: 
        hnr_score = Inharmonicity()(frequencies, magnitudes)
    ###
    
    onset_rate_score = OnsetRate()(audio44k)
    brightness_score = SpectralCentroidTime()(audio44k)
    dynamic_complexity_score, _ = DynamicComplexity()(audio16k)
    
    mel_bands = get_mel_bands(audio44k)
    novelty_curve = NoveltyCurve()(mel_bands)
    novelty_score = np.median(np.abs(np.diff(novelty_curve)))
    
    beats_loudness, beats_loudness_band_ratio = BeatsLoudness(beats=beat_positions)(audio44k)
    beatogram = Beatogram()(beats_loudness, beats_loudness_band_ratio)
    time_signature = Meter()(beatogram)
    
    #_, _, saliences, hum_starts, hum_ends = HumDetector()(audio44k)
    #hum_intervals = [(hum_start, hum_end, salience) for hum_start, hum_end, salience in zip(hum_starts, hum_ends, saliences)]
    
    features = {
        'Danceability': danceability_score[0],
        'Loudness': loudness_score,
        'BPM': bpm,
        'Key': key,
        'Key Scale': scale,
        'Energy': energy_score,
        'Chords Significance': chords_significance,
        'Inharmonicity': hnr_score,
        'Timbre': np.mean(mfcc_coeffs),
        'Onset Rate': onset_rate_score[1],
        'Brightness': brightness_score,
        'Dynamic Complexity': dynamic_complexity_score,
        'Novelty': novelty_score,
        'Time Signature': time_signature,
        #'Hum Intervals': hum_intervals
    }
    return features

In [526]:
def extract_audio_features(audio_file):
    # Load the audio file
    audio44k = MonoLoader(filename=audio_file)()
    audio16k = MonoLoader(filename=audio_file, sampleRate=16000)()

    # Run algorithms
    startTime = time.perf_counter()
    algorithm_features = run_essentia_algorithms(audio44k, audio16k)
    print(f"Algos: {time.perf_counter() - startTime}")
    
    startTime = time.perf_counter()
    spectrogram = mp3_to_spectrogram(audio_file, 22050)
    print(f"Spectro: {time.perf_counter() - startTime}")

    # Merge results
    features = algorithm_features | {'Spectrogram': spectrogram}
    return features

#### Main Code

In [527]:
# Class constructed from song path
# Song path must follow this format: /some/path/(int)^(video id)^(title).mp3
#                               e.g  /some/path/0^LlWGt_84jpg^Special Breed.mp3
class SongPath:
    def __init__(self, song_path: str):
        self.path = song_path
        self.filename = os.path.basename(song_path)

        song_filename_split = self.filename.split('^')
        if len(song_filename_split) != 3:
            raise Exception("The song's filename doesn't follow the correct format: /some/path/(int)^(video id)^(title).mp3")
        
        self.index, self.video_id, self.title_with_extension = song_filename_split

        self.index = int(self.index)
        self.title = os.path.splitext(self.title_with_extension)[0]

    def __str__(self):
        return f"Idx: {self.index},  videoID: {self.video_id}, title: {self.title_with_extension}"

In [528]:
def process_song(args):
    startTime = time.perf_counter()
    song_path, shared_songs_data, lock, process_times = args
    song = SongPath(song_path)

    # Extract song features
    song_features = extract_audio_features(song.path)

    with lock:
        songs_data_df = shared_songs_data['data']
        for feature, value in song_features.items():
            if feature not in songs_data_df.columns and isinstance(value, (tuple, set, list, np.ndarray, dict)):
                songs_data_df[feature] = np.nan
                songs_data_df[feature] = songs_data_df[feature].astype(object)
            songs_data_df.at[song.index, feature] = value
        shared_songs_data['data'] = songs_data_df
        end_time = time.perf_counter() - startTime
        idx = 0
        while idx < len(process_times) and process_times[idx] != 0:
            idx += 1
        process_times[idx] = end_time

In [529]:
def process_songs():
    manager = multiprocessing.Manager()
    lock = manager.Lock()
    songs_count = 20

    songs_data_full = songs_data.copy(deep=True)
    shared_songs_data = manager.dict({'data': songs_data_full})
        
    process_times = manager.Array('d', [0]*songs_count)
    song_paths = [os.path.join(DOWNLOAD_FOLDER, song_filename) for song_filename in os.listdir(DOWNLOAD_FOLDER)]

    args = [(song_path, shared_songs_data, lock, process_times) for song_path in song_paths]
    args = args[:songs_count]

    with multiprocessing.Pool(CPU_THREADS) as pool:
        pool.map(process_song, args)

    return shared_songs_data['data'], process_times

In [530]:
songs_data_full2, process_times2 = process_songs()
print(f"Mean: {np.mean(process_times2)}")
print(f"Median: {np.median(process_times2)}")
songs_data_full2

Algos: 7.0852239610012475
Spectro: 2.433535527999993
Algos: 8.771099337998749
Algos: 8.911475830998825
Algos: 9.481847647000905
Spectro: 3.4065044110011513
Spectro: 3.6054521730002307
Spectro: 3.507158641999922
Algos: 13.783791256000768
Spectro: 4.319706270000097
Algos: 9.572432539000147
Algos: 7.055353777999699
Algos: 7.654728404999332
Algos: 9.456548272999498
Spectro: 2.923060400000395
Spectro: 3.168906150998737
Spectro: 3.8768143449997297
Algos: 21.769178347998604
Spectro: 3.217884839999897
Spectro: 6.778353792999042
Algos: 7.087489554000058
Algos: 12.817030731001068
Spectro: 2.3754355700002634
Algos: 9.365769693000402
Spectro: 4.033976511000219
Algos: 10.575429423999594
Spectro: 3.2567175250005675
Algos: 11.508727584001463
Algos: 9.293771566000942
Spectro: 3.9600281530001666
Spectro: 3.013014830999964
Spectro: 4.217359104000934
Algos: 9.318549719999282
Algos: 7.747580385001129
Spectro: 2.7627607750000607
Algos: 6.6137369080006465
Spectro: 2.594818638999641
Spectro: 2.43209494500115

,title,artist,views,videoID,duration,Danceability,Loudness,BPM,Key,Key Scale,Energy,Chords Significance,Inharmonicity,Timbre,Onset Rate,Brightness,Dynamic Complexity,Novelty,Time Signature,Spectrogram
0,Special Breed,PolyCulture,34,LlWGt_84jpg,331,1.365855,5662.711426,128.004791,F,minor,399511.09375,"{'A': 1368, 'Ab': 44, 'Am': 1210, 'C': 363, 'C...",0.288343,-19.498598,4.38148,1235.493164,4.430042,176.945801,4.0,"[[-80.0, -59.258526, -42.22901, -38.483425, -3..."
1,Unnoticeable,Lost Ambitions,2,TD3za_a4uWo,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Time Dawdles Immersion,Happy Moppy Puppy,19,LlyA3VwQGfk,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Justness,Generallykoi,1,J14sCvTWh3Q,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,INTRANSIGEANCE,BFV,47,uAjBGvZFLi4,228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101040,Misery,Maroon 5,10005663,qHCHVHkTpbo,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101041,What Lies Beneath,Breaking Benjamin,10005140,KuSPuDFHY4s,215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101042,Yeh Dil Deewana,"Sonu Nigam, Hema Sardesai, Shankar Mahadevan",10005017,d7MDzmchS50,426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101043,Work (Freemasons Radio Edit),Kelly Rowland,10004642,vUj62L2fS1c,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [531]:
df = songs_data_full[songs_data_full['Danceability'].notna()]
df2 = songs_data_full2[songs_data_full2['Danceability'].notna()]

In [532]:
feature = "Spectrogram"
print(songs_data_full.iloc[100003][feature])
print(songs_data_full2.iloc[100003][feature])

[[-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]
 ...
 [-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]]
[[-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]
 ...
 [-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]
 [-80. -80. -80. ... -80. -80. -80.]]
